# RAG Workshop - Part 3: Agentic RAG

In this notebook, we'll build an AI agent that can:
- **Reason** through complex questions
- **Use tools** to search and check prerequisites
- **Plan** multi-step solutions
- **Show its thinking** (the ReAct pattern)

This is the difference between a simple chain (fixed steps) and an agent (dynamic reasoning).

## Setup

In [38]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
assert os.getenv("GOOGLE_API_KEY"), "Please set GOOGLE_API_KEY"
print("✅ Environment loaded!")

✅ Environment loaded!


---

## 1. Load Course Data

We'll use structured course data for the agent to work with.

In [39]:
# Load structured course data
with open("../data/courses.json", "r") as f:
    course_data = json.load(f)

courses = {c["code"]: c for c in course_data["courses"]}
prereq_chains = course_data["prerequisite_chains"]

print("Loaded courses:")
for code, info in courses.items():
    prereqs = ", ".join(info["prerequisites"]) or "None"
    print(f"  {code}: {info['name']} (prereqs: {prereqs})")

Loaded courses:
  CS101: Introduction to Programming (prereqs: None)
  CS201: Data Structures and Algorithms (prereqs: CS101)
  CS301: Introduction to Machine Learning (prereqs: CS201, MATH201, STAT101)
  CS401: Deep Learning (prereqs: CS301)
  CS501: Natural Language Processing (prereqs: CS401, MATH201)
  MATH101: Calculus I (prereqs: None)
  MATH201: Linear Algebra (prereqs: MATH101)
  STAT101: Introduction to Statistics and Probability (prereqs: MATH101)


---

## 2. Build the Vector Store (Same as before)

In [40]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Load and chunk documents
loader = DirectoryLoader("../data/syllabi", glob="**/*.md", loader_cls=TextLoader,
                         loader_kwargs={"encoding": "utf-8"})
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

# Create vector store
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embeddings, collection_name="agent_demo")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print(f"✅ Vector store ready with {len(chunks)} chunks")

✅ Vector store ready with 86 chunks


---

## 3. Define Agent Tools

Tools are capabilities the agent can use. Each tool is a function with a description.

In [41]:
from langchain.tools import tool

@tool
def search_courses(query: str) -> str:
    """Search for courses by topic, name, or description. 
    Use this to find courses related to a topic or keyword.
    """
    docs = retriever.invoke(query)
    results = []
    for doc in docs:
        source = Path(doc.metadata['source']).stem
        results.append(f"[{source}]: {doc.page_content[:200]}...")
    return "\n\n".join(results)

@tool
def get_course_info(course_code: str) -> str:
    """Get detailed information about a specific course by its code.
    Example: get_course_info("CS301")
    """
    code = course_code.upper().strip()
    if code in courses:
        c = courses[code]
        prereqs = ", ".join(c["prerequisites"]) or "None"
        return f"""Course: {c['code']} - {c['name']}
Credits: {c['credits']}
Prerequisites: {prereqs}
Instructor: {c['instructor']}
Semester: {c['semester']}
Level: {c['level']}
Topics: {', '.join(c['topics'])}
Description: {c['description']}"""
    return f"Course {code} not found."

@tool
def check_prerequisites(course_code: str) -> str:
    """Check the full prerequisite chain for a course.
    This shows ALL prerequisites needed, including indirect ones.
    Example: check_prerequisites("CS401")
    """
    code = course_code.upper().strip()
    
    def get_all_prereqs(course, visited=None):
        if visited is None:
            visited = set()
        if course in visited or course not in courses:
            return []
        visited.add(course)
        direct = courses[course]["prerequisites"]
        all_prereqs = list(direct)
        for prereq in direct:
            all_prereqs.extend(get_all_prereqs(prereq, visited))
        return all_prereqs
    
    if code not in courses:
        return f"Course {code} not found."
    
    direct = courses[code]["prerequisites"]
    all_prereqs = list(set(get_all_prereqs(code)))
    
    result = f"Prerequisites for {code} ({courses[code]['name']}):\n"
    result += f"  Direct prerequisites: {', '.join(direct) or 'None'}\n"
    result += f"  All prerequisites (including indirect): {', '.join(sorted(all_prereqs)) or 'None'}"
    
    return result

@tool
def find_learning_path(completed_courses: str, target_course: str) -> str:
    """Find the courses needed to reach a target course, given completed courses.
    
    Args:
        completed_courses: Comma-separated list of completed course codes (e.g., "CS101, MATH101")
        target_course: The course code you want to take (e.g., "CS401")
    """
    # Parse completed courses
    completed = set(c.strip().upper() for c in completed_courses.split(",") if c.strip())
    target = target_course.upper().strip()
    
    if target not in courses:
        return f"Target course {target} not found."
    
    def get_all_prereqs(course, visited=None):
        if visited is None:
            visited = set()
        if course in visited or course not in courses:
            return []
        visited.add(course)
        result = []
        for prereq in courses[course]["prerequisites"]:
            result.extend(get_all_prereqs(prereq, visited))
            result.append(prereq)
        return result
    
    # Get all prerequisites for target
    all_needed = get_all_prereqs(target) + [target]
    
    # Remove duplicates while preserving order
    seen = set()
    ordered = []
    for course in all_needed:
        if course not in seen:
            seen.add(course)
            ordered.append(course)
    
    # Filter out completed courses
    remaining = [c for c in ordered if c not in completed]
    
    if not remaining:
        return f"You've completed all prerequisites! You can take {target} now."
    
    result = f"Learning path from your current courses to {target}:\n"
    result += f"  Completed: {', '.join(sorted(completed)) or 'None'}\n"
    result += f"  Still needed (in order): {' → '.join(remaining)}"
    
    return result

@tool
def list_all_courses() -> str:
    """List all available courses with their basic information."""
    result = "Available courses:\n"
    for code, info in sorted(courses.items()):
        prereqs = ", ".join(info["prerequisites"]) or "None"
        result += f"  {code}: {info['name']} (Level: {info['level']}, Prereqs: {prereqs})\n"
    return result

# Collect all tools
tools = [search_courses, get_course_info, check_prerequisites, find_learning_path, list_all_courses]

print("✅ Tools defined:")
for t in tools:
    print(f"  - {t.name}: {t.description[:60]}...")

✅ Tools defined:
  - search_courses: Search for courses by topic, name, or description. 
    Use ...
  - get_course_info: Get detailed information about a specific course by its code...
  - check_prerequisites: Check the full prerequisite chain for a course.
    This sho...
  - find_learning_path: Find the courses needed to reach a target course, given comp...
  - list_all_courses: List all available courses with their basic information....


---

## 4. Create the ReAct Agent

The agent uses the **ReAct pattern**: Reasoning + Acting in a loop.

We use **LangGraph's** `create_react_agent` - the modern approach for building agents in LangChain ecosystem.

In [42]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0
)

# System prompt to make the agent more proactive
system_prompt = """You are a helpful course advisor for Fictional University.
When students ask about courses or learning paths, ALWAYS use your tools to look up information.
Don't ask clarifying questions - instead, search for relevant courses and provide recommendations.

For questions about AI, machine learning, or research paths:
1. First search for relevant courses using search_courses
2. Then check prerequisites using check_prerequisites  
3. Finally, suggest a learning path using find_learning_path

Be proactive and helpful - provide complete answers using your tools."""

# Create the ReAct agent with system prompt
agent_executor = create_agent(llm, tools, system_prompt=system_prompt)

def run_agent(question: str):
    """Run agent and display the reasoning process step by step."""
    print(f"Question: {question}")
    print("=" * 60)
    
    # Use stream to see each step of the agent's reasoning
    for step in agent_executor.stream({"messages": [("user", question)]}):
        # Model step - shows tool calls or final answer
        if "model" in step:
            messages = step["model"].get("messages", [])
            for msg in messages:
                if hasattr(msg, "tool_calls") and msg.tool_calls:
                    for tool_call in msg.tool_calls:
                        print(f"\n🤔 Thought: Using tool '{tool_call['name']}'")
                        print(f"   Args: {tool_call['args']}")
                elif hasattr(msg, "content") and msg.content:
                    print(f"\n{'='*60}")
                    print("FINAL ANSWER:")
                    print(msg.content)
        
        # Tool response step - shows observation
        if "tools" in step:
            messages = step["tools"].get("messages", [])
            for msg in messages:
                if hasattr(msg, "content"):
                    content = msg.content
                    truncated = content[:300] + "..." if len(content) > 300 else content
                    print(f"\n📋 Observation: {truncated}")

print("✅ Agent ready!")

✅ Agent ready!


---

## 5. Test the Agent

Watch the agent **think through** complex questions.

Pay attention to:
- **Thought**: The agent's reasoning
- **Action**: Which tool it chooses to use
- **Observation**: What the tool returns
- The agent **loops** until it has enough information

In [43]:
# Simple question first
run_agent("What is CS301 about?")

Question: What is CS301 about?

🤔 Thought: Using tool 'get_course_info'
   Args: {'course_code': 'CS301'}

📋 Observation: Course: CS301 - Introduction to Machine Learning
Credits: 4
Prerequisites: CS201, MATH201, STAT101
Instructor: Dr. Emily Watson
Semester: Fall
Level: Advanced
Topics: supervised learning, unsupervised learning, regression, classification, neural networks, model evaluation
Description: Introduction t...

FINAL ANSWER:
CS301 is an advanced, 4-credit course called Introduction to Machine Learning, taught by Dr. Emily Watson in the Fall semester. Prerequisites are CS201, MATH201, STAT101. The course covers supervised and unsupervised learning, regression, classification, neural networks, and model evaluation, with a focus on both theory and practical implementation.


In [36]:
# Now a question that requires checking prerequisites
run_agent("Can I take CS401 Deep Learning if I've only completed CS101?")

Question: Can I take CS401 Deep Learning if I've only completed CS101?

🤔 Thought: Using tool 'find_learning_path'
   Args: {'target_course': 'CS401', 'completed_courses': 'CS101'}

📋 Observation: Learning path from your current courses to CS401:
  Completed: CS101
  Still needed (in order): CS201 → MATH101 → MATH201 → STAT101 → CS301 → CS401

FINAL ANSWER:
No, you cannot directly take CS401 after only completing CS101. The learning path shows you still need to complete CS201, MATH101, MATH201, STAT101, and CS301 before you are eligible to take CS401.


### The Main Demo Question

This is the complex question that shows the agent's full reasoning capabilities.

In [44]:
# Complex multi-step question
run_agent("I've completed CS101 and MATH101. I want to do AI research. What courses should I take and in what order?")

Question: I've completed CS101 and MATH101. I want to do AI research. What courses should I take and in what order?

🤔 Thought: Using tool 'search_courses'
   Args: {'query': 'AI'}

📋 Observation: [CS401]: Building on the machine learning foundations from CS301, this course dives deep into the theory and practice of neural networks. Students will implement networks from scratch to understand the fundame...

[CS401]: Building on the machine learning foundations from CS301, this course dives de...

🤔 Thought: Using tool 'search_courses'
   Args: {'query': 'machine learning'}

📋 Observation: [CS301]: ## Course Description

This course provides a comprehensive introduction to machine learning, covering both theoretical foundations and practical applications. Students will learn the fundamental algo...

[CS301]: ## Course Description

This course provides a comprehensive introduction to m...

🤔 Thought: Using tool 'search_courses'
   Args: {'query': 'deep learning'}

📋 Observation: [CS401]:

### More Complex Queries

In [45]:
# Finding courses by topic
run_agent("What courses cover neural networks and who teaches them?")

Question: What courses cover neural networks and who teaches them?

🤔 Thought: Using tool 'search_courses'
   Args: {'query': 'neural networks'}

📋 Observation: [CS301]: ### Module 7: Neural Networks Introduction (Weeks 12-13)
- Perceptrons and multi-layer networks
- Activation functions
- Backpropagation basics
- Introduction to deep learning frameworks

### Module 8...

[CS301]: ### Module 7: Neural Networks Introduction (Weeks 12-13)
- Perceptrons and mu...

FINAL ANSWER:
I found that CS301 covers neural networks. I do not have information about instructors.


In [46]:
# Planning question
run_agent("I'm interested in Natural Language Processing. I've only taken high school math. What's the complete path I need to follow?")

Question: I'm interested in Natural Language Processing. I've only taken high school math. What's the complete path I need to follow?

🤔 Thought: Using tool 'search_courses'
   Args: {'query': 'Natural Language Processing'}

📋 Observation: [CS501]: ## Course Description

This graduate-level course provides comprehensive coverage of natural language processing (NLP), from classical techniques to modern deep learning approaches. Students will lear...

[CS501]: ## Course Description

This graduate-level course provides comprehensive cove...

🤔 Thought: Using tool 'check_prerequisites'
   Args: {'course_code': 'CS501'}

📋 Observation: Prerequisites for CS501 (Natural Language Processing):
  Direct prerequisites: CS401, MATH201
  All prerequisites (including indirect): CS101, CS201, CS301, CS401, MATH101, MATH201, STAT101

🤔 Thought: Using tool 'find_learning_path'
   Args: {'completed_courses': '', 'target_course': 'CS501'}

📋 Observation: Learning path from your current courses to CS501:


---

## 6. Interactive Demo

Try your own questions!

In [ ]:
# Interactive function that uses the same run_agent approach
# Try it with different questions!

# run_agent("What math courses do I need for machine learning?")
# run_agent("Compare CS301 and CS401 in terms of difficulty and topics")
# run_agent("I finished CS201 and MATH201. What can I take next semester?")

In [ ]:
# Uncomment and modify to try your own question!
# ask_agent("YOUR QUESTION HERE")

---

## Summary: Chain vs Agent

| Aspect | Simple RAG Chain | Agentic RAG |
|--------|------------------|-------------|
| **Flow** | Fixed: Retrieve → Generate | Dynamic: Reason → Act → Observe → Repeat |
| **Tools** | Just retriever | Multiple specialized tools |
| **Reasoning** | None | Shows thinking process |
| **Complex queries** | Often fails | Breaks down into steps |
| **Latency** | Fast (single pass) | Slower (multiple steps) |
| **Cost** | Lower (fewer LLM calls) | Higher (reasoning loop) |

### When to Use Each

**Use Simple RAG when:**
- Questions are straightforward
- Speed is critical
- Cost is a concern

**Use Agentic RAG when:**
- Questions require multi-step reasoning
- You need to combine multiple data sources
- Quality is more important than speed

---

## What's Next?

### LangChain Ecosystem

```
┌─────────────────────────────────────────────────────────┐
│              LANGCHAIN ECOSYSTEM (2026)                  │
├─────────────────────────────────────────────────────────┤
│                                                          │
│  langchain-core     → Base abstractions (LLMs, prompts) │
│  langchain          → Chains, RAG helpers, utilities    │
│  langchain.agents   → Agent framework (what we used!)   │ ← TODAY
│  langgraph          → Stateful graphs, production agents│ ← NEXT STEP
│  langsmith          → Observability, evals, debugging   │ ← PRODUCTION
│  langserve          → Deploy as REST API                │
│                                                          │
└─────────────────────────────────────────────────────────┘
```

### Further Learning

- **LangGraph**: When you need more control over agent flow
- **LangSmith**: For debugging and evaluating agents in production
- **Graph RAG**: For relationship-heavy data (Neo4j)
- **RAGAS**: For measuring RAG quality